# Question 1

Write a script that performs the following tasks using the airports.shp

* **Select those airports that are located in “Nome Census Area” print out their NAME and LOCID. (5 pt.)**

In [1]:
import arcpy
workspace = r"C:\Geog503\lab6"
arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True
fc = "airports.shp"
fields = arcpy.ListFields(fc)
for field in fields:
    print(field.name)

FID
Shape
AREA
PERIMETER
AIRPRTX020
LOCID
FEATURE
NAME
TOT_ENP
STATE
COUNTY
FIPS
STATE_FIPS


In [2]:
import arcpy
workspace = r"C:\Geog503\lab6"
arcpy.env.workspace = workspace
fc = "airports.shp"
fields = ["COUNTY", "NAME", "LOCID"]
cursor = arcpy.da.SearchCursor(fc, fields)
for row in cursor:
    if row[0] == "Nome Census Area":
        print("NAME = {:20} LOCID = {}".format(row[1], row[2]))

NAME = Koyuk                LOCID = KKA
NAME = Saint Michael        LOCID = 5S8
NAME = Stebbins             LOCID = WBB
NAME = Unalakleet           LOCID = UNK
NAME = Shaktoolik           LOCID = 38A
NAME = Elim                 LOCID = ELI
NAME = Golovin              LOCID = GLV
NAME = White Mountain       LOCID = WMO
NAME = Nome                 LOCID = OME
NAME = Savoonga             LOCID = SVA
NAME = Teller               LOCID = K54
NAME = Brevig Mission       LOCID = KTS
NAME = Shishmaref           LOCID = SHH
NAME = Wales                LOCID = IWK
NAME = Gambell              LOCID = GAM


*	**Update the STATE field of those selected airports to “AK” if their STATE field is empty. (5 pt.)**

In [3]:
import arcpy
workspace = r"C:\Geog503\lab6"
arcpy.env.workspace = workspace
fc = "airports.shp"
fieldname = "COUNTY"
delimfield = arcpy.AddFieldDelimiters(fc,fieldname)
where_clause = delimfield + "='Nome Census Area'"
print(where_clause)

fields = ["STATE"]
cursor =  arcpy.da.UpdateCursor(fc, fields, where_clause)
for row in cursor:
    if row[0].strip() == "":
        row[0] = "AK"
        cursor.updateRow(row)
del cursor

"COUNTY"='Nome Census Area'


*	**Export the information about those airports in “Nome Census Area” County to a text file, including these fields: NAME, LOCID, Longitude, and Latitude. (10 pt.)**

In [4]:
import arcpy
fc = r"C:\geog503\lab6\airports.shp"
txt = r"C:\geog503\lab6\Results\Nome_Census_Area.txt"
fieldname = "COUNTY"
delimfield = arcpy.AddFieldDelimiters(fc,fieldname)
where_clause = delimfield + "='Nome Census Area'"
# print(where_clause)

fields = ["NAME", "LOCID", "SHAPE@X", "SHAPE@Y"]
f = open(txt, 'w')

cursor =  arcpy.da.UpdateCursor(fc, fields, where_clause)
for row in cursor:
    line = "NAME: {:20} LOCID: {:10} X: {:20} Y: {:20} ".format(row[0], row[1], str(row[2]), str(row[3]))
    print(line)
    f.write(line + "\n")
            
del cursor
f.close()

NAME: Koyuk                LOCID: KKA        X: -161.151077271       Y: 64.9380722046        
NAME: Saint Michael        LOCID: 5S8        X: -162.110366821       Y: 63.4900512695        
NAME: Stebbins             LOCID: WBB        X: -162.282745361       Y: 63.5159187317        
NAME: Unalakleet           LOCID: UNK        X: -160.798950195       Y: 63.8883590698        
NAME: Shaktoolik           LOCID: 38A        X: -161.202529907       Y: 64.3626327515        
NAME: Elim                 LOCID: ELI        X: -162.270065308       Y: 64.6140060425        
NAME: Golovin              LOCID: GLV        X: -163.039520264       Y: 64.5434341431        
NAME: White Mountain       LOCID: WMO        X: -163.41255188        Y: 64.6891937256        
NAME: Nome                 LOCID: OME        X: -165.445251465       Y: 64.5121994019        
NAME: Savoonga             LOCID: SVA        X: -170.492630005       Y: 63.6863937378        
NAME: Teller               LOCID: K54        X: -166.3360137

# Question 2

Write a script that converts the airports.txt into a Point-type shapefile. The resulting shapefile should have the following fields based on the information from the airports.txt, including NAME, LOCID, X, and Y.  (20 pt.)

In [5]:
import arcpy
import os
workspace =  r"C:\Geog503\lab6\Results"
infile = r"C:\Geog503\lab6\airports.txt"
prj =  r"C:\Geog503\lab6\airports.prj"

arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True

pointList =[]
fc = os.path.splitext(os.path.basename(infile))[0] + ".shp"
print(fc)

f = open(infile)
for line in f.readlines():
    list1 = line.split(", ")
    point = []
    for item in list1:
        subitem = item.split(": ")
        point.append(subitem[1].strip('\n'))
    pointList.append(point)

spatialRef = arcpy.SpatialReference(prj)
arcpy.CreateFeatureclass_management(workspace,fc,"Point",spatial_reference=spatialRef)

arcpy.AddField_management(fc,"Name","string",field_length=50)
arcpy.AddField_management(fc,"LOCID","string",field_length=10)
arcpy.AddField_management(fc,"X","float")
arcpy.AddField_management(fc,"Y","float")

fields = ["Id","NAME","LOCID","X","Y","SHAPE@X","SHAPE@Y"]
cursor = arcpy.da.InsertCursor(fc, fields)

for point in pointList:
    id = int(point[0])
    name = point[1]
    locid = point[2]
    x = float(point[3])
    y = float(point[4])
    cursor.insertRow([id,name,locid,x,y,x,y])
del cursor

airports.shp
